In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load Data
class ClaimsDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.claims = data['claim'].tolist()
        self.abstracts = data['abstract'].tolist()
        self.labels = data['label'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.claims)

    def __getitem__(self, idx):
        claim = str(self.claims[idx])
        abstract = str(self.abstracts[idx])
        label = self.labels[idx]

        # Tokenize the claim and abstract together
        encoding = self.tokenizer(
            claim, abstract,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load Dataset
def load_data(filepath):
    return pd.read_csv(filepath)

# Parameters
model_name = "allenai/scibert_scivocab_uncased"
max_length = 256
batch_size = 4
num_epochs = 100
learning_rate = 1e-5

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Load your dataset
data = load_data("/kaggle/input/augclaimrag1/AugClaimWithRAG1.csv")  
dataset = ClaimsDataset(data, tokenizer, max_length)

# Split into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir= None,  # Logs will be saved locally in this directory
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="no",
    load_best_model_at_end=False,
    learning_rate=learning_rate,
    metric_for_best_model="accuracy",
    report_to=[],  # Disable reporting to external loggers (e.g., wandb)
)

# Trainer
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./scibert_claims_model")
tokenizer.save_pretrained("./scibert_claims_model")


In [ ]:
# Extract Training History
train_logs = trainer.state.log_history
train_accuracies = [log["eval_accuracy"] for log in train_logs if "eval_accuracy" in log]
train_epochs = list(range(1, len(train_accuracies) + 1))

# Plot Training and Validation Accuracy
plt.figure(figsize=(8, 5))
plt.plot(train_epochs, train_accuracies, label='Validation Accuracy', marker='o')
plt.title("Training and Validation Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.savefig("SciBERT_AUG_1_accuracy.png")  # Save the plot
plt.show()